In [50]:
# %pip install selenium
# %pip install beautifulsoup4
# %pip install webdriver-manager
# %pip install tabulate

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import random
import json
import os

In [2]:
base_url = "https://batdongsan.com.vn"
list_page_base_url = f"{base_url}/nha-dat-ban-ha-noi"

### Cấu hình Webdriver để tối ưu hoá chống chặn bot

In [3]:
options = Options()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)


In [ ]:

data = []
processed_links = set()
page_number = 1
max_pages = 150 
file_path = "batdongsan_data.json" # Đặt tên file ở ngoài để dễ truy cập

In [ ]:
if os.path.exists(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    print(f"Đã tải lại {len(data)} tin đăng từ file cũ.")
    for item in data:
        if 'link' in item:
            processed_links.add(item['link'])

Đã tải lại 2924 tin đăng từ file cũ.


In [ ]:
def classify_by_url(link):
    link = str(link).lower()
    
    if 'ban-can-ho-chung-cu' in link or 'ban-condotel' in link:
        return 'Căn hộ chung cư'
    
    elif 'ban-dat' in link:
        return 'Đất nền/Thổ cư'
    
    elif 'ban-nha-biet-thu-lien-ke' in link or 'biet-thu' in link or 'shophouse' in link or 'villa' in link:
        return 'Biệt thự/Liền kề'
    
    elif 'ban-nha-rieng' in link or 'ban-nha-mat-pho' in link:
        return 'Nhà riêng/Khác'
    
    else:
        return 'Khong_Ro' 

while page_number <= max_pages:
    if page_number == 1:
        current_page_url = list_page_base_url
    else:
        current_page_url = f"{list_page_base_url}/p{page_number}"

    print(f"--- Đang cào trang danh sách #{page_number}: {current_page_url} ---")
    
    try:
        driver.get(current_page_url)
        time.sleep(random.uniform(5, 8)) 

        list_html = driver.page_source
        list_soup = BeautifulSoup(list_html, "html5lib")

        listings = list_soup.find_all('div', class_='js__card')
        
        if not listings:
            print(f"Không tìm thấy tin đăng nào trên trang {page_number}. Đã đạt đến trang cuối hoặc bị chặn. Dừng lại.")
            break 
        
        page_links = []
        

        for listing in listings:
            link_tag = listing.find('a', class_='js__product-link-for-product-id')
            relative_link = link_tag['href'] if link_tag and 'href' in link_tag.attrs else None
            
            if relative_link:
                full_link = f"{base_url}{relative_link}"
                if full_link not in processed_links:
                    processed_links.add(full_link)
                    
                    item_data = {}
                    item_data['title'] = listing.find('span', class_='pr-title').get_text(strip=True) if listing.find('span', class_='pr-title') else 'N/A'
                    item_data['price'] = listing.find('span', class_='re__card-config-price').get_text(strip=True) if listing.find('span', class_='re__card-config-price') else 'N/A'
                    
                    area_tag = listing.find('span', class_='re__card-config-area')
                    item_data['area'] = area_tag.get_text(strip=True).replace('·', '').replace('m²', ' m²') if area_tag else 'N/A'
                    
                    bedroom_tag = listing.find('span', class_='re__card-config-bedroom')
                    item_data['bedroom'] = bedroom_tag.find('span').get_text(strip=True) + ' PN' if bedroom_tag and bedroom_tag.find('span') else 'N/A'
                    
                    toilet_tag = listing.find('span', class_='re__card-config-toilet')
                    item_data['toilet'] = toilet_tag.find('span').get_text(strip=True) + ' WC' if toilet_tag and toilet_tag.find('span') else 'N/A'
                    
                    item_data['link'] = full_link
                    page_links.append(item_data)
            

        for item_data in page_links:
            full_link = item_data['link']
            
            item_data['address'] = 'N/A'
            item_data['legal'] = 'N/A'
            item_data['furniture'] = 'N/A'
            item_data['post_date'] = 'N/A'
            item_data['loai_bds'] = classify_by_url(full_link)

            try:
                driver.get(full_link)
                print(f"  > Đang truy cập chi tiết: {full_link}")
                time.sleep(random.uniform(3, 5)) 

                detail_html = driver.page_source
                detail_soup = BeautifulSoup(detail_html, "html5lib")
                
                # Trích xuất Ngày đăng
                post_date_div = detail_soup.find('div', class_='re__pr-short-info-item js__pr-config-item')
                if post_date_div:
                    title_span = post_date_div.find('span', class_='title')
                    value_span = post_date_div.find('span', class_='value')
                    if title_span and 'Ngày đăng' in title_span.get_text(strip=True):
                        item_data['post_date'] = value_span.get_text(strip=True) if value_span and value_span.get_text(strip=True) else 'N/A'
                
                # Trích xuất Địa chỉ chi tiết
                address_tag = detail_soup.select_one('span.re__pr-short-description.js__pr-address')
                if address_tag:
                    item_data['address'] = address_tag.get_text(strip=True)
                    
                # Trích xuất Pháp lý, Nội thất, Loại hình
                spec_tags = detail_soup.find_all('div', class_='re__pr-specs-content-item')
                for spec in spec_tags:
                    label_tag = spec.find('span', class_='re__pr-specs-content-item-title')
                    value_tag = spec.find('span', class_='re__pr-specs-content-item-value')
                    if label_tag and value_tag:
                        label = label_tag.get_text(strip=True)
                        value = value_tag.get_text(strip=True)
                        if 'Pháp lý' in label:
                            item_data['legal'] = value
                        elif 'Nội thất' in label:
                            item_data['furniture'] = value
            except Exception as e:
                print(f"  > Lỗi khi truy cập {full_link}: {e}")

        data.extend(page_links) 
        
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        
        print(f"  >>> Đã lưu dữ liệu tạm thời. Tổng số tin hiện tại: {len(data)}")

        page_number += 1
        
    except Exception as e:
        print(f"Lỗi chung khi xử lý trang danh sách {page_number}: {e}")
        break 
        
driver.quit()

print("\n==============================")
print(f"Hoàn thành cào dữ liệu. Tổng số tin đăng cuối cùng: {len(data)}")
print("==============================")
print(f"Dữ liệu cuối cùng đã được lưu vào: {file_path}")

### 2. Hiển thị dữ liệu thu thập được, lưu lại trong file .CSV

In [ ]:
import json
import pandas as pd
from tabulate import tabulate  
from datetime import datetime
import re

In [ ]:
with open("batdongsan_data.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)

cols = ['price', 'area', 'bedroom', 'toilet', "address", 'legal', 'furniture', 'post_date', 'loai_bds']

print("\n 5 tin Bất Động Sản đầu tiên:\n")
print(tabulate(df[cols].head(), headers='keys', tablefmt='grid', showindex=False))

print(f"\n Tổng số tin: {len(df):,}")

csv_file = "batdongsan.csv"
df.to_csv(csv_file, index=False, encoding='utf-8-sig', sep=';')
print(f"\nĐã lưu dữ liệu thành công vào file: {csv_file}")



 5 tin Bất Động Sản đầu tiên:

+----------------+----------+-----------+----------+-----------------------------------------------------------------+----------------+----------------+-------------+------------------+
| price          | area     | bedroom   | toilet   | address                                                         | legal          | furniture      | post_date   | loai_bds         |
+================+==========+===========+==========+=================================================================+================+================+=============+==================+
| Giá thỏa thuận | 63,1  m² | 2 PN      | 2 WC     | Dự án Vinhomes Ocean Park Gia Lâm, Xã Dương Xá, Gia Lâm, Hà Nội | Sổ đỏ/ Sổ hồng | Cơ bản         | 18/10/2025  | Căn hộ chung cư  |
+----------------+----------+-----------+----------+-----------------------------------------------------------------+----------------+----------------+-------------+------------------+
| 2 tỷ           | 100  m²  | N/A     